##Sélection du questionnaire à générer
- le numéro de phase correspond à celui du fichier de combinaisons

In [127]:
phase=3

###Importations

- **yaml** pour la gestion des tableaux pour les exercices
- **codecs** pour la gestion de l'UTF8 dans les fichiers

In [128]:
import yaml,codecs
import random as rd

###Gestion des associations entre stimuli et URL

In [129]:
shareLien='<a href="https://soundcloud.com/linguistique-bordeaux/%s">son/a>'
embedLien='<iframe width="100%%" height="166" scrolling="no" frameborder="no" src="https://w.soundcloud.com/player/?url=https%%3A//api.soundcloud.com/tracks/%s&amp;color=ff5500&amp;auto_play=false&amp;hide_related=false&amp;show_comments=true&amp;show_user=true&amp;show_reposts=false"></iframe>'

In [130]:
import csv
shareLiens={}
embedLiens={}
with open('../../../Copy/Recherche/Rugby/Test2/StimuliLiens.csv', 'rb') as csvfile:
    associations = csv.reader(csvfile, delimiter=';', quotechar='"')
    for association in associations:
        shareLiens[(association[0],association[1],association[2],association[3])]=shareLien%association[4]
        if len(association)<6: 
            numero="000000000"
        else:
            numero=association[5]
        embedLiens[(association[0],association[1],association[2],association[3])]=embedLien%numero

###Chargement du module eCloze/MoodleXML.py

- le module **imp** permet de charger un module python directement à partir de l'adresse du fichier

In [131]:
import imp
moodle=imp.load_source('MoodleXML', '../../eCloze/MoodleXML.py')

###Définition des deux fois trois séries de stimuli

- Trois pour le sport :
 1. Rugby
 1. Foot
 1. Course
- Trois pour le non-sport :
 1. Journal télévisé
 1. Magazine sportif
 1. Soirée électorale 2007

In [132]:
noms={"Sport":["Rugby","Foot","Course"],"Autre":["Journal","Magazine","Election"]}

###Définition des oppositions

In [133]:
grille={"Sport":["Sport","Autre"],"Description":["Description","Commentaire"],"Specifique":["Specifique","Generique"]}
oppositions={"Description":["Description","Commentaire"],"Specifique":["Specifique","Generique"]}

###Définition des consignes
- les consignes Sport et Desc ont deux champs à remplir pour les URL des stimuli
- la consigne Uniques a un champ à remplir pour l'URL du stimuli

In [134]:
consigneSport=  u'''<table align="center" border="0">
                <tbody>
                    <tr>
                        <td>%s</td>
                        <td>%s</td>
                    </tr>
                    <tr>
                        <td style="text-align: center;">1</td>
                        <td style="text-align: center;">2</td>
                    </tr>
                </tbody>
            </table>
            <p>Écoutez les deux extraits. (Attention de ne pas cliquer sur Play another track !)</p>
            <p>Après les avoir écoutés, sélectionnez celui qui vous semble correspondre à du commentaire sportif.<br />
            {1:MCV:=1~=2}</p>'''

consigneDesc=  u'''<table align="center" border="0">
                <tbody>
                    <tr>
                        <td>%s</td>
                        <td>%s</td>
                    </tr>
                    <tr>
                        <td style="text-align: center;">1</td>
                        <td style="text-align: center;">2</td>
                    </tr>
                </tbody>
            </table>
            <p>Écoutez les deux extraits. (Attention de ne pas cliquer sur Play another track !)</p>
            <p>Après les avoir écoutés, sélectionnez celui qui vous semble correspondre à de la description directe.<br />
            {1:MCV:=1~=2}</p>'''

consigneUniques=  u'''<table align="center" border="0">
                <tbody>
                    <tr>
                        <td>%s</td>
                    </tr>
                    <tr>
                        <td style="text-align: center;">1</td>
                    </tr>
                </tbody>
            </table>
            <p>Écoutez l'extrait. (Attention de ne pas cliquer sur Play another track !)</p>
            <p>Après avoir écouté, répondez aux deux questions.<br />
            S'agit-il d'un commentaire sportif ? : {1:MCV:=Oui~=Non}<br />
            Est-ce que le locuteur semble décrire directement ce qu'il voit ? : {1:MCV:=Oui~=Non}</p>'''

###Gestion de l'ordre de présentation des paires
- quatre paires pour le sport
- quatre paires pour les commentaires

random.shuffle mélange l'intérieur de la liste (attention, la fonction renvoie **None**)

In [135]:
ordreSport=[(0,1),(0,1),(1,0),(1,0)]
ordreDesc=[(0,1),(0,1),(1,0),(1,0)]
rd.shuffle(ordreSport)
rd.shuffle(ordreDesc)
print ordreSport
print ordreDesc

[(1, 0), (0, 1), (1, 0), (0, 1)]
[(0, 1), (0, 1), (1, 0), (1, 0)]


###Gestion des combinaisons

In [136]:
fichierCombinaisons='combinaisons'+str(phase)+'.yaml'
with open(fichierCombinaisons, 'r') as yaml_file:
    questions=yaml.load(yaml_file)
#for question in questions:
#    print question
#    for el in questions[question]:
#        print el

###Génération d'un questionnaire

In [137]:
questionnaire=moodle.MoodleXML()

uniquesClozes=moodle.XMLClozes("Perception/Questionnaire%s/Absolu"%phase)
for element in questions["Uniques"]:
    elStim=(element[0],element[1],element[2],element[3])
    lien=embedLiens[elStim]
    titre=element[0][0]+element[1][0]+element[2][0]+element[3][0]
    uniquesClozes.addExercice(moodle.ClozeExercice(titre,consigneUniques%(lien)))
questionnaire.addQuiz(uniquesClozes.getClozes())

In [138]:
def makeSigle(quadruplet):
    return quadruplet[0][0]+quadruplet[1][0]+quadruplet[2][0]+quadruplet[3][0]

In [139]:
sportClozes=moodle.XMLClozes("Perception/Questionnaire%s/Sport"%phase)
for i,element in enumerate(questions["Sport"]):
    (autre,sport)=element
    sport1=element[ordreSport[i][0]]
    sport2=element[ordreSport[i][1]]
    elStimSport1=(sport1[0],sport1[1],sport1[2],sport1[3])
    elStimSport2=(sport2[0],sport2[1],sport2[2],sport2[3])
    lienSport1=embedLiens[elStimSport1]
    lienSport2=embedLiens[elStimSport2]
    titre=makeSigle(elStimSport1)+" vs "+makeSigle(elStimSport2)
    print titre
    sportClozes.addExercice(moodle.ClozeExercice(titre,consigneSport%(lienSport1,lienSport2)))
questionnaire.addQuiz(sportClozes.getClozes())

FSDS vs MADS
EACS vs RSCS
CSDG vs EADG
MACG vs RSCG


In [140]:
descClozes=moodle.XMLClozes("Perception/Questionnaire%s/Description"%phase)
for i,element in enumerate(questions["Description"]):
    (desc,comm)=element
    desc1=element[ordreDesc[i][0]]
    desc2=element[ordreDesc[i][1]]
    elStimDesc1=(desc1[0],desc1[1],desc1[2],desc1[3])
    elStimDesc2=(desc2[0],desc2[1],desc2[2],desc2[3])
    lienDesc1=embedLiens[elStimDesc1]
    lienDesc2=embedLiens[elStimDesc2]
    titre=makeSigle(elStimDesc1)+" vs "+makeSigle(elStimDesc2)
    print titre
    descClozes.addExercice(moodle.ClozeExercice(titre,consigneDesc%(lienDesc1,lienDesc2)))
questionnaire.addQuiz(descClozes.getClozes())

CSDS vs CSCS
FSDG vs FSCG
MACS vs EADS
JACG vs JADG


###Écriture du questionnaire au format Moodle XML

In [141]:
fichierXML='questionnaire'+str(phase)+'.xml'
with codecs.open(fichierXML, 'w',"utf8") as output:
    output.write(questionnaire.getXML())